# Querying Metadata with the REST API

In this notebook we explore searching for metadata from the REST API. The REST API provides a method to programmatically extract a JSON representation of the meta data from the API.

First we load some python dependancies that we will use as part of this notebook and set the variable `ENDPOINT_URL` to the location of the REST API.

In [58]:
import pandas as pd
import yaml
import requests
import dask
import intake
import dask.dataframe as dd

def load_config() -> dict:
    """Helper function to load the configuration file"""
    with open('config.yml') as file_handle:
        config = yaml.load(file_handle, yaml.FullLoader)
    return config

config = load_config()
ENDPOINT_URL = config['rest_api']

Below we print the location of the endpoint we will query in this notebook:

In [43]:
print(f"REST API Endpoint: {ENDPOINT_URL}")

REST API Endpoint: https://ada-sam-app.oxfordfun.com/json


## Querying Shots with the REST API

We're going to use the python `requests` library to query metadata from get metadata from the database. All we need to do to get a result is to query the database with a HTTP GET at the appropriate endpoint. For example, to get information about different experimental shots we can query the `/json/shots/` endpoint.

In [97]:
response = requests.get(f'{ENDPOINT_URL}/shots/')
result = response.json()
print(f"Query returned status code: {response.status_code}")

Query returned status code: 200


The shots endpoint returns a JSON payload with a lot of different information inside it. Let's take a closer look at what is returned:

In [84]:
result.keys()

dict_keys(['items', 'total', 'page', 'size', 'pages', 'column_metadata'])

The main item of interest is the `items` entry. This contains a list of meta-data items that correspond to our query. In this case, each item contains information about a different MAST shot. Each item lots of information about different shots, for example the shot ID, the campaign the shot was part of, the pre- and post-shot description by investigators.

For more information on the what's returned by the API you can look at the endpoint documentation:

https://ada-sam-app.oxfordfun.com/redoc

Let's take a look at the first item returned:


In [100]:
result['items'][0]

{'shot_id': 47206,
 'timestamp': '2023-03-31T16:04:44',
 'preshot_description': 'retry',
 'postshot_description': 'rtp stops st07 and st25, plus event 40 and 50 (px-cc-soft and trip) at 0.867s. Signs of BC action in xcm/ibc-* but it only made a slight difference to the LVPS dc link voltage (about 10V).',
 'campaign': 'MU3',
 'reference_shot': None,
 'scenario': None,
 'heating': None,
 'pellets': False,
 'rmp_coil': False,
 'current_range': None,
 'divertor_config': None,
 'plasma_shape': None,
 'comissioner': 'UKAEA',
 'facility': 'MAST',
 'cpf_p03249': None,
 'cpf_p04673': None,
 'cpf_p04674': None,
 'cpf_p04675': None,
 'cpf_p04676': None,
 'cpf_p04677': None,
 'cpf_p04678': None,
 'cpf_p04679': None,
 'cpf_p04680': None,
 'cpf_p04681': None,
 'cpf_p04833': None,
 'cpf_p04993': None,
 'cpf_p05007': None,
 'cpf_p05008': None,
 'cpf_p05009': None,
 'cpf_p05010': None,
 'cpf_p05011': None,
 'cpf_p05015': None,
 'cpf_p05016': None,
 'cpf_p05017': None,
 'cpf_p05025': None,
 'cpf_p05027'

Of course, we can read all this JSON data directly into common python data analysis packages, for example, we can create a `pandas` dataframe for the meta data. 

In [113]:
df = pd.DataFrame(result['items'])
df.head()

,shot_id,timestamp,preshot_description,postshot_description,campaign,reference_shot,scenario,heating,pellets,rmp_coil,...,cpf_vol_ipmax,cpf_vol_max,cpf_vol_truby,cpf_wmhd_ipmax,cpf_wmhd_max,cpf_wmhd_truby,cpf_zeff_ipmax,cpf_zeff_max,cpf_zeff_truby,cpf_zmag_efit
0,47206,2023-03-31T16:04:44,retry,"rtp stops st07 and st25, plus event 40 and 50 ...",MU3,NaN,NaN,None,False,False,...,None,None,None,None,None,None,None,None,None,None
1,47205,2023-03-31T15:52:21,"Brake chopper test. Restore 44539. P1, TF and ...",abort,MU3,44539.0,NaN,None,False,False,...,None,None,None,None,None,None,None,None,None,None
2,47204,2023-03-31T15:38:20,Repeat daily test shot with PS enable windows ...,Error in channels 11 and 12 on bolometer. Wave...,MU3,NaN,NaN,None,False,False,...,None,None,None,None,None,None,None,None,None,None
3,47203,2023-03-31T15:19:43,Restore prepared setup all-coil-first-shot. Ex...,"Looks OK, I don't have a code to test the magn...",MU3,NaN,NaN,None,False,False,...,None,None,None,None,None,None,None,None,None,None
4,47202,2023-03-31T14:59:33,retry,"Ran OK, no trips, all PF currents as demanded.",MU3,45847.0,NaN,None,False,False,...,None,None,None,None,None,None,None,None,None,None


### Searching & Filtering Data

All REST API endpoints can take query parameters to filter the data returned.

TODO

### Pagination 
The REST API responses are _paginated_, meaning that only a subset of the full items are returned with each query. Pagination is used to limit the total number of requests made by each user to prevent any single user overloading the server with huge data requests.

Pagination information is included in the response:

 - `total` - this is the total number of items in the database which matched your query.
 - `size` - the number of items returned on each page.
 - `page` - the current page of information. Each page contains `size` number of items
 - `pages` - the total number of pages for information in the database. This is `math.ceil(size / total)`.

You can control the pagination using the following options:
 - `page=xx` to the next page of items.
 - `size=xx` to set the number of results returned by each page.

In [90]:
response = requests.get(f'{ENDPOINT_URL}/shots/?size=2&page=2')
result = response.json()

print(f"Number of items returned = {len(result['items'])}")
result

Number of items returned = 2


{'items': [{'shot_id': 47204,
   'timestamp': '2023-03-31T15:38:20',
   'preshot_description': 'Repeat daily test shot with PS enable windows tight around the current waveforms, as a test for diagnostic noise. Enable as many diagnostics as possible.',
   'postshot_description': 'Error in channels 11 and 12 on bolometer. Waveforms mostly OK except Px chopped early for no apparent reason (no RTP events)',
   'campaign': 'MU3',
   'reference_shot': None,
   'scenario': None,
   'heating': None,
   'pellets': False,
   'rmp_coil': False,
   'current_range': None,
   'divertor_config': None,
   'plasma_shape': None,
   'comissioner': 'UKAEA',
   'facility': 'MAST',
   'cpf_p03249': None,
   'cpf_p04673': None,
   'cpf_p04674': None,
   'cpf_p04675': None,
   'cpf_p04676': None,
   'cpf_p04677': None,
   'cpf_p04678': None,
   'cpf_p04679': None,
   'cpf_p04680': None,
   'cpf_p04681': None,
   'cpf_p04833': None,
   'cpf_p04993': None,
   'cpf_p05007': None,
   'cpf_p05008': None,
   'cpf_p

### Loading Paginated Responses with Dask

Some times we want to load all of the results from all pages of of a query. The next code snippet shows how to lazily build a query for each page of information and return a `dask` dataframe which will hold all of the results.

In [108]:
response = requests.get(f'{ENDPOINT_URL}/shots/')
num_pages = response.json()['pages']

@dask.delayed
def load_one_page(i): 
    """Load one page of requests from the REST API"""
    response = requests.get(f'{ENDPOINT_URL}/shots/?page={i}')
    response = response.json()
    # Important! need to include column_metadata here to get the correct data types for each column
    return (
        pd.DataFrame
            .from_dict(response['items'])
            .astype(response['column_metadata'])
    )

df = dd.from_delayed([load_one_page(i) for i in range(1, 5)])
df

,shot_id,timestamp,preshot_description,postshot_description,campaign,reference_shot,scenario,heating,pellets,rmp_coil,current_range,divertor_config,plasma_shape,comissioner,facility,cpf_p03249,cpf_p04673,cpf_p04674,cpf_p04675,cpf_p04676,cpf_p04677,cpf_p04678,cpf_p04679,cpf_p04680,cpf_p04681,cpf_p04833,cpf_p04993,cpf_p05007,cpf_p05008,cpf_p05009,cpf_p05010,cpf_p05011,cpf_p05015,cpf_p05016,cpf_p05017,cpf_p05025,cpf_p05027,cpf_p05028,cpf_p05029,cpf_p05030,cpf_p05032,cpf_p05033,cpf_p05153,cpf_p06000,cpf_p06001,cpf_p06002,cpf_p06003,cpf_p06004,cpf_p10963,cpf_p10964,cpf_p12441,cpf_p12450,cpf_p12451,cpf_p12452,cpf_p15202,cpf_p15203,cpf_p15209,cpf_p15659,cpf_p15660,cpf_p15661,cpf_p20000,cpf_p20204,cpf_p20205,cpf_p20206,cpf_p20207,cpf_p20208,cpf_p21010,cpf_p21011,cpf_p21012,cpf_p21021,cpf_p21022,cpf_p21029,cpf_p21035,cpf_p21037,cpf_p21041,cpf_p21042,cpf_p21043,cpf_p21044,cpf_p21045,cpf_p21046,cpf_p21047,cpf_p21048,cpf_p21051,cpf_p21052,cpf_p21053,cpf_p21054,cpf_p21055,cpf_p21056,cpf_p21075,cpf_p21076,cpf_p21077,cpf_p21078,cpf_p21079,cpf_p21080,cpf_p21081,cpf_p21082,cpf_p21083,cpf_p21084,cpf_p21085,cpf_p21086,cpf_p21087,cpf_p21088,cpf_p21089,cpf_p21092,cpf_p21093,cpf_abort,cpf_amin_ipmax,cpf_amin_max,cpf_amin_truby,cpf_area_ipmax,cpf_area_max,cpf_area_truby,cpf_bepmhd_ipmax,cpf_bepmhd_max,cpf_bepmhd_truby,cpf_betmhd_ipmax,cpf_betmhd_max,cpf_betmhd_truby,cpf_bt_ipmax,cpf_bt_max,cpf_bt_truby,cpf_c2ratio,cpf_column_temp_in,cpf_column_temp_out,cpf_creation,cpf_dwmhd_ipmax,cpf_dwmhd_max,cpf_dwmhd_truby,cpf_enbi_max_ss,cpf_enbi_max_sw,cpf_exp_date,cpf_exp_number,cpf_exp_time,cpf_gdc_duration,cpf_gdc_time,cpf_ibgas_pressure,cpf_ip_av,cpf_ip_max,cpf_jnbi_ipmax,cpf_jnbi_ipmax_ss,cpf_jnbi_ipmax_sw,cpf_jnbi_max,cpf_jnbi_max_ss,cpf_jnbi_max_sw,cpf_jnbi_total,cpf_jnbi_total_ss,cpf_jnbi_total_sw,cpf_jnbi_truby,cpf_jnbi_truby_ss,cpf_jnbi_truby_sw,cpf_johm_ipmax,cpf_johm_max,cpf_johm_total,cpf_johm_truby,cpf_kappa_ipmax,cpf_kappa_max,cpf_kappa_truby,cpf_li_2_ipmax,cpf_li_2_max,cpf_li_2_truby,cpf_li_3_ipmax,cpf_li_3_max,cpf_li_3_truby,cpf_log_base_pressure,cpf_ndl_co2_ipmax,cpf_ndl_co2_max,cpf_ndl_co2_truby,cpf_ne0_ipmax,cpf_ne0_max,cpf_ne0_truby,cpf_ne0ratio_ipmax,cpf_ne0ruby,cpf_ne_bar_ipmax,cpf_ne_yag_bar_ipmax,cpf_ngreenwald_ipmax,cpf_ngreenwaldratio_ipmax,cpf_o2ratio,cpf_objective,cpf_pe0_ipmax,cpf_pe0_max,cpf_pe0_truby,cpf_pe0ruby,cpf_pic,cpf_pnbi_ipmax,cpf_pnbi_ipmax_ss,cpf_pnbi_ipmax_sw,cpf_pnbi_max,cpf_pnbi_max_ss,cpf_pnbi_max_sw,cpf_pnbi_truby,cpf_pnbi_truby_ss,cpf_pnbi_truby_sw,cpf_pohm_ipmax,cpf_pohm_max,cpf_pohm_truby,cpf_postshot,cpf_prad_ipmax,cpf_prad_max,cpf_prad_truby,cpf_pradne2,cpf_preshot,cpf_program,cpf_pulno,cpf_q95_ipmax,cpf_q95_min,cpf_q95_truby,cpf_reference,cpf_rgeo_ipmax,cpf_rgeo_max,cpf_rgeo_truby,cpf_rinner_da,cpf_rinner_efit,cpf_rmag_efit,cpf_router_da,cpf_router_efit,cpf_sarea_ipmax,cpf_sarea_max,cpf_sarea_truby,cpf_sl,cpf_summary,cpf_tamin_max,cpf_tarea_max,cpf_tautot_ipmax,cpf_tautot_max,cpf_tautot_truby,cpf_tbepmhd_max,cpf_tbetmhd_max,cpf_tbt_max,cpf_tdwmhd_max,cpf_te0_ipmax,cpf_te0_max,cpf_te0_truby,cpf_te0ratio_ipmax,cpf_te0ruby,cpf_te_yag_bar_ipmax,cpf_tend,cpf_tend_ibgas,cpf_tend_nbi,cpf_tend_nbi_ss,cpf_tend_nbi_sw,cpf_term_code,cpf_tftend,cpf_tftstart,cpf_tipmax,cpf_tkappa_max,cpf_tli_2_max,cpf_tli_3_max,cpf_tndl_co2_max,cpf_tne0_max,cpf_tpe0_max,cpf_tpnbi_max,cpf_tpnbi_max_ss,cpf_tpnbi_max_sw,cpf_tpohm_max,cpf_tprad_max,cpf_tq95_min,cpf_trgeo_max,cpf_truby,cpf_tsarea_max,cpf_tstart,cpf_tstart_ibgas,cpf_tstart_nbi,cpf_tstart_nbi_ss,cpf_tstart_nbi_sw,cpf_ttautot_max,cpf_tte0_max,cpf_tvol_max,cpf_twmhd_max,cpf_tzeff_max,cpf_useful,cpf_vol_ipmax,cpf_vol_max,cpf_vol_truby,cpf_wmhd_ipmax,cpf_wmhd_max,cpf_wmhd_truby,cpf_zeff_ipmax,cpf_zeff_max,cpf_zeff_truby,cpf_zmag_efit
npartitions=4,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

When we are ready, we can compute the all queries and return the resulting dataframe with all the meta data gathered from the database.

In [110]:
df.compute()

,shot_id,timestamp,preshot_description,postshot_description,campaign,reference_shot,scenario,heating,pellets,rmp_coil,...,cpf_vol_ipmax,cpf_vol_max,cpf_vol_truby,cpf_wmhd_ipmax,cpf_wmhd_max,cpf_wmhd_truby,cpf_zeff_ipmax,cpf_zeff_max,cpf_zeff_truby,cpf_zmag_efit
0,47206,2023-03-31T16:04:44,retry,"rtp stops st07 and st25, plus event 40 and 50 ...",MU3,<NA>,<NA>,<NA>,False,False,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,47205,2023-03-31T15:52:21,"Brake chopper test. Restore 44539. P1, TF and ...",abort,MU3,44539.0,<NA>,<NA>,False,False,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,47204,2023-03-31T15:38:20,Repeat daily test shot with PS enable windows ...,Error in channels 11 and 12 on bolometer. Wave...,MU3,<NA>,<NA>,<NA>,False,False,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,47203,2023-03-31T15:19:43,Restore prepared setup all-coil-first-shot. Ex...,"Looks OK, I don't have a code to test the magn...",MU3,<NA>,<NA>,<NA>,False,False,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,47202,2023-03-31T14:59:33,retry,"Ran OK, no trips, all PF currents as demanded.",MU3,45847.0,<NA>,<NA>,False,False,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,47011,2023-01-21T09:54:47,[Integrated control test] Restore IntegratedCt...,Terminated early but the gas response was bett...,MU2,46994.0,15.0,Ohmic,False,False,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
46,47010,2023-01-21T09:31:38,[TF test] Restore All_Coil_First_Shot.,Currents look good.,MU2,<NA>,<NA>,<NA>,False,False,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
47,47009,2023-01-20T17:57:31,"[FPP-02 TAEs] Repeat, diagnostics in position 1.",Good SW but only 30ms of south. Not useful for...,MU2,47008.0,14.0,<NA>,False,False,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
48,47008,2023-01-20T17:38:16,[FPP-02 TAEs] Straight repeat again. Diagnosti...,Density higher than the previous shot early on...,MU2,47007.0,14.0,2 Beams,False,False,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


## Querying with File Streaming API

### Using Intake

In [ ]:
catalog = intake.open_catalog(f'{ENDPOINT_URI}/meta_catalog.yml')
gui = intake.interface.gui.GUI([catalog])
gui

Reading from the Intake catalog

In [ ]:
catalog.signal_datasets.read()

Direct access with the a known URI.

In [ ]:
pd.read_parquet(f'{ENDPOINT_URI}/files/shots')

Or even read the file in CSV format

In [ ]:
pd.read_csv(f'{ENDPOINT_URI}/files/shots?format=csv')